In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

In [26]:
def _read_oecd_metadata_from_file():
    rows = pd.read_excel('ReadMe_ICIO2023_extended.xlsx', 
                        sheet_name='RowItems',
                        skiprows=3,
                        index_col=0,
                        usecols=lambda x: 'Unnamed' not in x)
    rows = rows.set_index('Sector code')
    cols = pd.read_excel('ReadMe_ICIO2023_extended.xlsx',
                    sheet_name='ColItems',
                    skiprows=3,
                    index_col=0,
                    usecols=lambda x: 'Unnamed' not in x)
    cols=cols.set_index('Sector code')
    indices = dict()
    true_indices = np.where(rows.index.isna())[0]
    indices["Z"] = rows.iloc[:true_indices[0]]
    indices["Z"] = indices['Z'][['Country', 'Industry/Final demand']].apply(tuple, axis=1).to_dict()
    indices["rows_tls"] = rows.iloc[true_indices[0]+1:true_indices[1]]
    indices["rows_VA"] = rows.iloc[true_indices[1]+1:]

    true_indices = np.where(cols.index.isna())[0]
    indices["cols_FD"] = cols.iloc[true_indices[0]+1:-1]
    indices["cols_FD"] = indices['cols_FD'][['Country', 'Industry/Final demand']].apply(tuple, axis=1).to_dict()
    return indices

In [27]:
meta = _read_oecd_metadata_from_file()

C:\Users\andreasst\Miniconda3\lib\site-packages\openpyxl\reader\workbook.py:118: UserWarning: Print area cannot be set to Defined name: ColItems!$A:$E.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
C:\Users\andreasst\Miniconda3\lib\site-packages\openpyxl\reader\workbook.py:118: UserWarning: Print area cannot be set to Defined name: RowItems!$A:$E.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
C:\Users\andreasst\Miniconda3\lib\site-packages\openpyxl\reader\workbook.py:118: UserWarning: Print area cannot be set to Defined name: ColItems!$A:$E.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
C:\Users\andreasst\Miniconda3\lib\site-packages\openpyxl\reader\workbook.py:118: UserWarning: Print area cannot be set to Defined name: RowItems!$A:$E.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")


In [ ]:
meta['Z']

In [19]:
path = Path('/Users/andreasst/work/raw_v2023')
year = 1995
#oecd = parse_new_oecd(path, 2000)

path = os.path.abspath(os.path.normpath(str(path)))


# determine which oecd file to be parsed
if not os.path.isdir(path):
    # 1. case - one file specified in path
    oecd_file = path
    path = os.path.split(oecd_file)[0]

oecd_file_list = [
        fl
        for fl in os.listdir(path)
        if (
            os.path.splitext(fl)[-1] in [".csv", ".CSV", ".zip"]
            and os.path.splitext(fl)[0]
            in [str(year)+oo for oo in [".SML", ""]]
        )
    ]
oecd_file = os.path.join(path, oecd_file_list[0])
oecd_file_name = os.path.split(oecd_file)[1]

oecd_raw = pd.read_csv(oecd_file, sep=",", index_col=0).fillna(0)

mon_unit = "Million USD"

oecd_totals_col = ["TOTAL", "OUT"]
oecd_totals_row = ["OUT", "OUTPUT"]

oecd_raw.drop(oecd_totals_col, axis=1, errors="ignore", inplace=True)
oecd_raw.drop(oecd_totals_row, axis=0, errors="ignore", inplace=True)

In [20]:
# Important - these must not match any country or industry name
VA = oecd_raw.filter(items=['VA', 'TLS'], axis=0)
TLS = oecd_raw.loc['TLS']
final_demand = oecd_raw.filter(
    regex="HFCE|NPISH|NPS|GGFC|GFCF|INVNT|INV|DIRP|DPABR|FD|P33|DISC", axis=1
)
factor_input = oecd_raw.filter(regex="VALU|TAX", axis=0)

In [10]:
factor_input

,ARG_A01_02,ARG_A03,ARG_B05_06,ARG_B07_08,ARG_B09,ARG_C10T12,ARG_C13T15,ARG_C16,ARG_C17_18,ARG_C19,...,ZAF_GGFC,ZAF_GFCF,ZAF_INVNT,ZAF_DPABR,ROW_HFCE,ROW_NPISH,ROW_GGFC,ROW_GFCF,ROW_INVNT,ROW_DPABR


In [21]:
Z = oecd_raw.loc[
    oecd_raw.index.difference(VA.index),
    oecd_raw.columns.difference(final_demand.columns),
]

Y = final_demand.loc[final_demand.index.difference(VA.index), :]

In [15]:
Y

,ARG_HFCE,ARG_NPISH,ARG_GGFC,ARG_GFCF,ARG_INVNT,ARG_DPABR,AUS_HFCE,AUS_NPISH,AUS_GGFC,AUS_GFCF,...,ZAF_GGFC,ZAF_GFCF,ZAF_INVNT,ZAF_DPABR,ROW_HFCE,ROW_NPISH,ROW_GGFC,ROW_GFCF,ROW_INVNT,ROW_DPABR
ARG_A01_02,3774.893122,0.0,0.000000,580.449257,-758.686421,0.000000,0.082705,0.000032,0.003908,0.037579,...,0.000000,0.39061,0.317617,0.102300,198.375377,0.031739,1.389620,6.146837,6.494394,1.268545
ARG_A03,191.063824,0.0,0.000000,0.000000,16.924082,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.007699,0.177707,0.000031,0.000239,0.000000,0.000043,0.108909
ARG_B05_06,430.987185,0.0,0.000000,0.000000,-84.889122,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000012,0.000000,0.002857,0.000000,0.000000,0.943175,1.239177,0.000000
ARG_B07_08,29.511350,0.0,0.000000,0.000000,48.483634,0.000000,0.000391,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,2.407261,0.000000,0.000000,0.026499,0.733038,0.000000
ARG_B09,0.000000,0.0,0.000000,692.153932,-2.009927,0.000000,0.000000,0.000000,0.000000,0.000012,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002693,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZAF_P,0.000000,0.0,0.000000,0.000000,0.000000,0.109054,0.000000,0.000000,0.000000,0.000000,...,7828.935124,0.00000,-0.084997,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.145890
ZAF_Q,0.000000,0.0,0.000000,0.000000,0.000000,0.220572,0.000000,0.000000,0.000000,0.000000,...,2540.164407,0.00000,-0.077075,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.169276
ZAF_R,0.108776,0.0,0.000011,0.004509,0.000000,0.535868,0.181955,0.001554,0.067079,0.001456,...,0.000000,0.00000,-11.006981,0.000000,0.488549,0.039146,0.216470,0.023539,0.000000,8.349695
ZAF_S,0.000000,0.0,0.000000,0.000000,0.000000,0.256455,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.411120,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.919272


In [22]:
Z.index = Z.index.map(meta['Z'])
Z

ARG_A01_02    ARG_A03   ARG_B05_06  ARG_B07_08     ARG_B09  \
ARG A01_02  4149.287356   8.522726     0.481234    0.000000    0.000000   
    A03        0.000000  37.659640     0.042927    0.000000    0.000000   
    B05_06     0.089622   0.002407   409.329338    0.000000    0.000000   
    B07_08     0.254013   0.110517    79.566170  251.567286    0.000000   
    B09        0.000000   0.000000  1019.411550   21.035357  105.755647   
...                 ...        ...          ...         ...         ...   
ZAF P          0.000000   0.000000     0.000000    0.000000    0.000000   
    Q          0.000000   0.000000     0.000000    0.000000    0.000000   
    R          0.000448   0.000000     0.000000    0.000000    0.000000   
    S          0.000000   0.000000     0.000000    0.000000    0.000000   
    T          0.000000   0.000000     0.000000    0.000000    0.000000   

             ARG_C10T12  ARG_C13T15     ARG_C16  ARG_C17_18      ARG_C19  ...  \
ARG A01_02  8091.070922  950.158509  133.507834   73.525378     5.698406  ...   
    A03      501.517776    0.000000    0.000000    0.030148     0.531107  ...   
    B05_06     0.247465    0.000000    0.000000    0.286708  4129.960914  ...   
    B07_08     3.210803    1.878463    0.006727    2.250891   143.903984  ...   
    B09        0.000000    0.000000    0.000000    0.000000     0.000000  ...   
...                 ...         ...         ...         ...          ...  ...   
ZAF P          0.000000    0.000000    0.000000    0.000000     0.000000  ...   
    Q          0.000000    0.000000    0.000000    0.000000     0.000000  ...   
    R          0.005075    0.000380    0.000006    0.000293     0.000000  ...   
    S          0.000000    0.000000    0.000000    0.000000     0.000000  ...   
    T          0.000000    0.000000    0.000000    0.000000     0.000000  ...   

                 ZAF_K       ZAF_L       ZAF_M       ZAF_N       ZAF_O  \
ARG A01_02    0.000000    0.003641    0.003138    0.006363    0.010294   
    A03       0.000000    0.000000    0.000000    0.000000    0.000000   
    B05_06    0.000000    0.000000    0.000000    0.000000    0.000000   
    B07_08    0.000000    0.000000    0.000000    0.000000    0.000000   
    B09       0.000000    0.000000    0.000000    0.000000    0.000000   
...                ...         ...         ...         ...         ...   
ZAF P        71.906711   10.234890   21.784761   13.045954   65.605207   
    Q       110.121704  419.103638  218.638083  174.888719  238.125842   
    R         7.255556   57.155009   88.522990   99.507467   17.985141   
    S         5.156561   75.303343   61.654302   43.585841    2.991038   
    T         0.000000    0.000000    0.000000    0.000000    0.000000   

                 ZAF_P      ZAF_Q       ZAF_R      ZAF_S  ZAF_T  
ARG A01_02    0.002687   0.010920    0.149967   0.248332    0.0  
    A03       0.000000   0.000000    0.000000   0.000000    0.0  
    B05_06    0.000000   0.000001    0.000008   0.000006    0.0  
    B07_08    0.000000   0.000000    0.000000   0.000000    0.0  
    B09       0.000000   0.000000    0.000000   0.000000    0.0  
...                ...        ...         ...        ...    ...  
ZAF P         2.936579   3.789715   22.058325  21.286506    0.0  
    Q       102.756337  14.779432  277.230626  39.824772    0.0  
    R        25.401818   6.047882  257.782606  25.808768    0.0  
    S         9.657098   5.540013   11.674120  25.807119    0.0  
    T         0.000000   0.000000    0.000000   0.000000    0.0  

[3645 rows x 3645 columns]

In [23]:
Z.columns = Z.columns.map(meta['Z'])

Z

ARG                                                  \
                 A01_02        A03       B05_06      B07_08         B09   
ARG A01_02  4149.287356   8.522726     0.481234    0.000000    0.000000   
    A03        0.000000  37.659640     0.042927    0.000000    0.000000   
    B05_06     0.089622   0.002407   409.329338    0.000000    0.000000   
    B07_08     0.254013   0.110517    79.566170  251.567286    0.000000   
    B09        0.000000   0.000000  1019.411550   21.035357  105.755647   
...                 ...        ...          ...         ...         ...   
ZAF P          0.000000   0.000000     0.000000    0.000000    0.000000   
    Q          0.000000   0.000000     0.000000    0.000000    0.000000   
    R          0.000448   0.000000     0.000000    0.000000    0.000000   
    S          0.000000   0.000000     0.000000    0.000000    0.000000   
    T          0.000000   0.000000     0.000000    0.000000    0.000000   

                                                                         ...  \
                 C10T12      C13T15         C16     C17_18          C19  ...   
ARG A01_02  8091.070922  950.158509  133.507834  73.525378     5.698406  ...   
    A03      501.517776    0.000000    0.000000   0.030148     0.531107  ...   
    B05_06     0.247465    0.000000    0.000000   0.286708  4129.960914  ...   
    B07_08     3.210803    1.878463    0.006727   2.250891   143.903984  ...   
    B09        0.000000    0.000000    0.000000   0.000000     0.000000  ...   
...                 ...         ...         ...        ...          ...  ...   
ZAF P          0.000000    0.000000    0.000000   0.000000     0.000000  ...   
    Q          0.000000    0.000000    0.000000   0.000000     0.000000  ...   
    R          0.005075    0.000380    0.000006   0.000293     0.000000  ...   
    S          0.000000    0.000000    0.000000   0.000000     0.000000  ...   
    T          0.000000    0.000000    0.000000   0.000000     0.000000  ...   

                   ZAF                                                  \
                     K           L           M           N           O   
ARG A01_02    0.000000    0.003641    0.003138    0.006363    0.010294   
    A03       0.000000    0.000000    0.000000    0.000000    0.000000   
    B05_06    0.000000    0.000000    0.000000    0.000000    0.000000   
    B07_08    0.000000    0.000000    0.000000    0.000000    0.000000   
    B09       0.000000    0.000000    0.000000    0.000000    0.000000   
...                ...         ...         ...         ...         ...   
ZAF P        71.906711   10.234890   21.784761   13.045954   65.605207   
    Q       110.121704  419.103638  218.638083  174.888719  238.125842   
    R         7.255556   57.155009   88.522990   99.507467   17.985141   
    S         5.156561   75.303343   61.654302   43.585841    2.991038   
    T         0.000000    0.000000    0.000000    0.000000    0.000000   

                                                               
                     P          Q           R          S    T  
ARG A01_02    0.002687   0.010920    0.149967   0.248332  0.0  
    A03       0.000000   0.000000    0.000000   0.000000  0.0  
    B05_06    0.000000   0.000001    0.000008   0.000006  0.0  
    B07_08    0.000000   0.000000    0.000000   0.000000  0.0  
    B09       0.000000   0.000000    0.000000   0.000000  0.0  
...                ...        ...         ...        ...  ...  
ZAF P         2.936579   3.789715   22.058325  21.286506  0.0  
    Q       102.756337  14.779432  277.230626  39.824772  0.0  
    R        25.401818   6.047882  257.782606  25.808768  0.0  
    S         9.657098   5.540013   11.674120  25.807119  0.0  
    T         0.000000   0.000000    0.000000   0.000000  0.0  

[3645 rows x 3645 columns]

In [24]:
Z.index.set_names(['regions', 'sectors'], inplace=True)
Z.columns.set_names(['regions', 'sectors'], inplace=True)

In [25]:
Z

regions                  ARG                                                  \
sectors               A01_02        A03       B05_06      B07_08         B09   
regions sectors                                                                
ARG     A01_02   4149.287356   8.522726     0.481234    0.000000    0.000000   
        A03         0.000000  37.659640     0.042927    0.000000    0.000000   
        B05_06      0.089622   0.002407   409.329338    0.000000    0.000000   
        B07_08      0.254013   0.110517    79.566170  251.567286    0.000000   
        B09         0.000000   0.000000  1019.411550   21.035357  105.755647   
...                      ...        ...          ...         ...         ...   
ZAF     P           0.000000   0.000000     0.000000    0.000000    0.000000   
        Q           0.000000   0.000000     0.000000    0.000000    0.000000   
        R           0.000448   0.000000     0.000000    0.000000    0.000000   
        S           0.000000   0.000000     0.000000    0.000000    0.000000   
        T           0.000000   0.000000     0.000000    0.000000    0.000000   

regions                                                                       \
sectors               C10T12      C13T15         C16     C17_18          C19   
regions sectors                                                                
ARG     A01_02   8091.070922  950.158509  133.507834  73.525378     5.698406   
        A03       501.517776    0.000000    0.000000   0.030148     0.531107   
        B05_06      0.247465    0.000000    0.000000   0.286708  4129.960914   
        B07_08      3.210803    1.878463    0.006727   2.250891   143.903984   
        B09         0.000000    0.000000    0.000000   0.000000     0.000000   
...                      ...         ...         ...        ...          ...   
ZAF     P           0.000000    0.000000    0.000000   0.000000     0.000000   
        Q           0.000000    0.000000    0.000000   0.000000     0.000000   
        R           0.005075    0.000380    0.000006   0.000293     0.000000   
        S           0.000000    0.000000    0.000000   0.000000     0.000000   
        T           0.000000    0.000000    0.000000   0.000000     0.000000   

regions          ...         ZAF                                      \
sectors          ...           K           L           M           N   
regions sectors  ...                                                   
ARG     A01_02   ...    0.000000    0.003641    0.003138    0.006363   
        A03      ...    0.000000    0.000000    0.000000    0.000000   
        B05_06   ...    0.000000    0.000000    0.000000    0.000000   
        B07_08   ...    0.000000    0.000000    0.000000    0.000000   
        B09      ...    0.000000    0.000000    0.000000    0.000000   
...              ...         ...         ...         ...         ...   
ZAF     P        ...   71.906711   10.234890   21.784761   13.045954   
        Q        ...  110.121704  419.103638  218.638083  174.888719   
        R        ...    7.255556   57.155009   88.522990   99.507467   
        S        ...    5.156561   75.303343   61.654302   43.585841   
        T        ...    0.000000    0.000000    0.000000    0.000000   

regions                                                                         
sectors                   O           P          Q           R          S    T  
regions sectors                                                                 
ARG     A01_02     0.010294    0.002687   0.010920    0.149967   0.248332  0.0  
        A03        0.000000    0.000000   0.000000    0.000000   0.000000  0.0  
        B05_06     0.000000    0.000000   0.000001    0.000008   0.000006  0.0  
        B07_08     0.000000    0.000000   0.000000    0.000000   0.000000  0.0  
        B09        0.000000    0.000000   0.000000    0.000000   0.000000  0.0  
...                     ...         ...        ...         ...        ...  ...  
ZAF     P         65